# Evaluate Vector Representation
- word vector가 아닌 sentence vector로 뉴스기사의 유사도를 비교합니다

1. 'news_sample_test.txt' 파일을 load한다. 이 때, 개별 뉴스기사의 첫번째 문장만 활용한다. 예를 들어, 1번 뉴스기사에 41개 문장이 있다면, 그 중 첫번째 문장만 선택한다.
2.   word2vec과 fastText 모델을 load한다
3. word vector로 sentence vector를 만든다.
  1. 문장을 단어(또는 토큰)단위로 나눈다.
  2. 단어의 embedding vector를 word2vec과 fastText 모델에서 불러온다.
  3. **문장의 모든 단어의 embedding vector값을 전부 합친다.**
  4. 합쳐준 값을 문장벡터로 간주한다.
4. 이제 각 뉴스기사의 요약의 embedding vector를 활용해서, 개별 뉴스기사가 서로 유사한지 비교한다(cosine similarity 사용)




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/'
filename = 'news_sample_test.txt'

### 전처리

In [ ]:
!pip install hanja
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu, mac 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

import re, hanja
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

EMAIL_PATTERN = re.compile(r'''(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))''', re.VERBOSE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.VERBOSE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        tokenizer = Kkma()
        
    return tokenizer

def tokenize(tokenizer_name, original_sent, pos=False):
    tokenizer = get_tokenizer(tokenizer_name)
    sentence = original_sent.replace('\n', '').strip()
    if pos:
        tokens = tokenizer.pos(sentence)
        tokens = [morph + "/" + tag for morph, tag in tokens]
    else:
      # tokenizer.nouns(sentence) -> 명사만 추출
        tokens = tokenizer.morphs(sentence)
        
    # tokenized_sent = ' '.join(post_process(tokens))
    tokenized_sent = ' '.join(tokens)
    
    return tokenized_sent

def cleansing_other(sentence: str = None) -> str:
    """
    문장을 전처리 (이메일, URL, 공백 등 제거) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub(EMAIL_PATTERN, ' ', sentence)
    sentence = re.sub(URL_PATTERN, ' ', sentence)
    sentence = re.sub(MULTIPLE_SPACES, ' ', sentence)
    sentence = sentence.replace(", )", "")
    
    return sentence

def cleansing_chinese(sentence: str = None) -> str:
    """
    한자를 변환하는 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    # chinese character를 앞뒤로 괄호가 감싸고 있을 경우, 대부분 한글 번역임
    sentence = re.sub("\([\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]+\)", "", sentence)
    # 다른 한자가 있다면 한글로 치환
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sentence) is not None:
        sentence = hanja.translate(sentence, 'substitution')

    return sentence

def cleansing_special(sentence: str = None) -> str:
    """
    특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence: str = None) -> str:
    """
    숫자를 전처리(delexicalization) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

def preprocess_sent(sentence: str = None) -> str:
    """
    모든 전처리를 수행 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sent_clean = sentence
    sent_clean = cleansing_other(sent_clean)
    sent_clean = cleansing_chinese(sent_clean)
    sent_clean = cleansing_special(sent_clean)
    sent_clean = cleansing_numbers(sent_clean)
    sent_clean = re.sub('\s+', ' ', sent_clean)
    sent_clean = tokenize('mecab', sent_clean)

    return sent_clean

## 1번
1. 'news_sample.txt' 파일을 load한다.
2. 이 때, 개별 뉴스기사의 첫 3개 문장을 활용한다. 예를 들어, 1번 뉴스기사에 41개 문장이 있다면, 그 중 1번부터 3번까지만 선택한다.
  - 이를 위해 `kss` 를 설치해야한다
3. 3개 문장을 전부 하나로 연결한다. 이 때, 띄어쓰기(' ')를 활용한다. 이를 뉴스의 요약이라고 간주한다
4. 각 뉴스기사의 요약을 전처리한다




In [ ]:
!pip install kss

In [ ]:
import kss

In [ ]:
news_summ = []
with open(os.path.join(path, filename), 'r', encoding='utf-8') as f:


## 2번

-   word2vec과 fastText 모델을 load한다


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
w2v_model = 

In [ ]:
ft_model = 

## 3번
- word vector로 sentence vector를 만든다.
  1. 문장을 단어(또는 토큰)단위로 나눈다.
  2. 단어의 embedding vector를 word2vec과 fastText 모델에서 불러온다.
  3. **문장의 모든 단어의 embedding vector값을 전부 합친다.**
  4. 합쳐준 값을 문장벡터로 간주한다. 합쳐진 벡터를 summ_vec라고 한다

In [ ]:
# 안녕 하 세요. -> (1,300), (1,300), (1,300) -> [3, 300]
# sentence = '안녕 하 세요' -> for loop -> 안녕 -> model['안녕'] -> 300차원 벡터 -> 하 model['하'] -> 300차원 벡터
# 300차원벡터를 append를 한다 

'''
sent_vec_list = 
[[a1, ..., a300],
[b1, ..., b300],
[c1, ..., c300]]

sent_vec =
[a1+b1+c1, ... , a300+b300+c300]

'''


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def get_sent_vector(model, sentence):
  sent_vec = []
  for word in sentence.split(' '):
    # out of vocabulary = oov 
    if word in model.wv.vocab.keys():
      sent_vec.append(model[word])
    # fastText
    # word: 써비스 -> model.vocab -> string distance -> 서비스
  return sent_vec

In [ ]:
# model.wv.vocab.keys()
# model['word'(변수명)]
def get_sent_vector(model, sentence):


In [ ]:
# summ_vec를 만든다
# np.summ(sent_vec, axis=0)


In [ ]:
df = pd.DataFrame(data=results, columns=['news_id','summ_vectors'])

## 4번

- 이제 각 뉴스기사의 요약의 embedding vector를 활용해서, 개별 뉴스기사가 서로 유사한지 비교한다(cosine similarity 사용)
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
